## Las correrías de Rafaela (I.P.) y Adolfo (H.P.)

Rafaela lleva semanas sintiendo que Adolfo la viene tratando diferente; con algo de desidia y con comportamientos inusuales. Por esto, usando un programa que compró en Unilago, ha descargado los datos de los últimos tres meses de las dos SIM-Cards de Adolfo (personal y corporativa), así como los extractos de su tarjeta de crédito (un día que él dejó su computador desatendido), y un registro de llamadas de la factura del celular.

En esta carpeta Usted encontrará los archivos:
* "evidencia/WApp_Corporativo.csv"
* "evidencia/WApp_Personal.csv"
* "evidencia/DatosFacturaMovistar.csv"
* "evidencia/Municipios.xls"
* "evidencia/TarjetaCred.csv"

### Episodio 1: 

#### ¿Será que habla con otra? ¿Cuánto lleva en esas?

"Este guache sí es desatento, pero no tanto. ¿Por qué anda tan alejado? Miremos el WhatsApp..."

##### (A) El WhatsApp Personal

In [ ]:
import pandas as pd
import seaborn as sb

file = "evidencia/WApp_Personal.csv"
WA_Person = pd.read_csv(file)

# Usar ".head()" o ".columns" para inspeccionar columnas
WA_Person.head()

In [ ]:
# Hay una columna de ruido (Solo importa "LastSeen"). Retirar:
WA_Person.drop( ["Unnamed: 0"], axis=1, inplace=True)
WA_Person.head()

In [ ]:
# Use ".dtypes" para asegurarse que los datos están en formato correco
WA_Person.dtypes

In [ ]:
# Tipo object es texto. Cambiar a "datetime"
WA_Person["LastSeen"] = pd.to_datetime(WA_Person["LastSeen"])
WA_Person.dtypes

##### Voila

Finalmente tenemos los datos limpios. Ahora ¿qué hacer con ellos? Con lo poco que hay, miremos a qué horas se conecta/desconecta Adolfo regularmente.

Para esto deberíamos tomar los objetos tipo DateTime de "LastSeen" y **separar la fecha de la hora**. Quizá **enviar la fecha al índice del DataFrame** y conservar una columna con las horas de desconexión.

In [ ]:
import datetime as dt

# Crear una columna tipo float con la hora en decimales (combinando horas y minutos)
WA_Person["Hora"] = pd.to_datetime(WA_Person["LastSeen"]).dt.hour + (1.0/60)*pd.to_datetime(WA_Person["LastSeen"]).dt.minute
WA_Person.head()

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Dibujar Hora de Desconexión contra Fecha
WA_Person.plot(x="LastSeen", y="Hora", style='.', color='b', legend=False)
plt.title("Hora de Desconexión VS. Fecha")
plt.show()

##### (B) El WhatsApp Corporativo

In [ ]:
file = "evidencia/WApp_Corporativo.csv"
WA_Corp = pd.read_csv(file)
WA_Corp.drop( ["Unnamed: 0"], axis=1, inplace=True)
WA_Corp["LastSeen"] = pd.to_datetime(WA_Corp["LastSeen"])
WA_Corp["Hora"] = pd.to_datetime(WA_Corp["LastSeen"]).dt.hour + (1.0/60)*pd.to_datetime(WA_Corp["LastSeen"]).dt.minute

ax1 = WA_Person.plot(x="LastSeen", y="Hora", style='.', color='b', legend=False)
WA_Corp.plot(x="LastSeen", y="Hora", style='.', color='r', ax=ax1, legend=False)

plt.title("Hora de Desconexión VS. Fecha (azul=personal; rojo=corporativo)")
plt.show()

"¡Este troglodita se pasó julio de puro trasnocho hablando con alguna suripanta por el WA de la oficina!"

### Episodio 2:

#### ¿Será solo coqueteo o están haciendo algo?

Miremos las llamadas

In [ ]:
file = "evidencia/DatosFacturaMovistar.csv"

Cel = pd.read_csv(file)
Cel.drop( ["Unnamed: 0"], axis=1, inplace=True)
Cel.head()

"Nada raro con los números; son todos de la familia."

[Llamada a un amigo]

Resulta que los códigos de las antenas funcionan así:

Si se tiene "WWXXXYYZZZ", entonces:

* 'WW': Departamento
* 'XXX': Municipio
* 'YY': Operador
* 'ZZZ': Localización antena

In [ ]:
# Remover columnas inútiles
Cel.drop( ["Duracion"], axis=1, inplace=True)

# Corroborar tipos
print(Cel.dtypes)
print("\n")

# Arreglar tipos
Cel["CodeAntena"] = Cel["CodeAntena"].astype(str)
print(Cel.dtypes)

# Tomar solo el pedazo de CodeAntena referente a Departamento y Municipio
Cel["DeptoMuni"] = Cel["CodeAntena"].apply(lambda x: x[:5])
Cel.head()

In [ ]:
# Cargae base de municipios para comparar
muni = pd.read_excel("evidencia/Municipios.xls")
muni.head()

In [ ]:
# El archivo original es un desastre; tomar nombres de columnas con base en la fila 2
muni.columns = muni.iloc[2,:]

# Alternativa a "head" (más general; indexa filas y columnas)
muni.iloc[0:7,:]

In [ ]:
# desechar filas basura y renombrar columnas
muni.drop( muni.index[[0,1,2,3,4,1125]], inplace=True )
muni = muni[["Código Deapartamento","Nombre Departamento","Código Municipio","Nombre Municipio"]]

reemplazo = {"Código Deapartamento":"Cod_Depto",
             "Nombre Departamento":"Depto",
             "Código Municipio":"Cod_Muni",
             "Nombre Municipio":"Muni"}

muni.rename(columns=reemplazo, inplace=True)
muni.head()

In [ ]:
# Convertir a string para comparar con Departamentos y Municipios del celular
muni["Cod_Depto"] = muni["Cod_Depto"].astype(str)
muni["Cod_Muni"] = muni["Cod_Muni"].astype(str)

# Crear columna comparable 
# (Tanto Cel como Muni quedan con un "DeptoMuni" con codigos en string)
muni["DeptoMuni"] = muni[["Cod_Depto","Cod_Muni"]].apply(lambda x: ''.join(x), axis=1)
muni.head()

In [ ]:
# Para cada Codigo "DeptoMuni" que aparezca en el Cel, acumular filas que coincidan en "DeptoMuni" de muni
lista = []
for dm in Cel["DeptoMuni"].unique():
    lista.append( muni[ muni["DeptoMuni"] == dm ] )

# Uso más común de concat
pd.concat(lista)

In [ ]:
# Ahora sí: muéstreme las llamadas que tenían origen en San Antero
Cel[ Cel["DeptoMuni"]=='23672' ]

"¡Pero este vergajo que hace en San Antero! 

Todo el verraco fin de semana del 3 al 5 de agosto que me dijo que se iba de seminario de la empresa.

¡Es que me va a oir!" 


### Episodio 3: ¡¿Y con la platica de nosotros?!

In [ ]:
file = "evidencia/TarjetaCred.csv"

# caragr datos extracto tarjeta
TCred = pd.read_csv(file)
TCred.drop( ["Unnamed: 0"], axis=1, inplace=True)
TCred.head()

Miremos a ver si es cuando se pierde los viernes que le da por hacer cosas raras.

In [ ]:
# Utilizar dias y meses en español como referencia
dias = {0:"1. Lun",1:"2. Mar",2:"3. Mie",3:"4. Jue",4:"5. Vie",5:"6. Sab",6:"7. Dom"}
meses = {6:"Jun",7:"Jul",8:"Ago",9:"Sep"}

# La columna fecha hora viene como string y queremos analizarla como DATETIME
TCred["Fecha/Hora"] = pd.to_datetime(TCred["Fecha/Hora"])

# Obtener día de la semana y mes a partir de objeto tipo DATETIME
# OJO: tanto weekday como month devuelven un entero (los mapeamos con "dias" y "meses")
TCred["DiaSemana"] = TCred["Fecha/Hora"].apply( lambda x: dias[ x.weekday() ] )
TCred["Mes"] = TCred["Fecha/Hora"].apply( lambda x: meses[ x.month ] )

TCred.head()

In [ ]:
# Pasar datetime a indice
TCred.set_index("Fecha/Hora", inplace=True)
TCred.head()

In [ ]:
# GROUPBY: Muy importante para análisis.
Aux = TCred.groupby(["Mes","DiaSemana"]).mean()
Aux

In [ ]:
# Stack y Unstack: importante al trabajar con multi-indices (i.e., dimensiones altas)
    # Stack es enviar columnas a indices
    # Unstack es enviar indices a columnas
Aux = Aux.unstack()
Aux

In [ ]:
Aux.columns = Aux.columns.droplevel()
Aux

In [ ]:
Aux.plot(kind='bar')
plt.title("Gasto PROMEDIO agrupado por 'dia de la semana' Y 'mes'")
plt.show()

"¡Cómo así que este tipo se estuvo gastando 200 o 300 mil pesos los martes y jueves de julio!

Muestre a ver en qué..."

In [ ]:
# MUY ÚTIL: slicing basado en valores (y lógica)
TCred[ TCred["Valor"] > 200000 ]

In [ ]:
# MUY ÚTIL: operaciones sobre columnas
TCred["Valor"][ TCred["Valor"] > 200000].sum()

"¡Pero este desgraciado se gasta 3 millones en un paseo con una aparecida, con los ahorritos de nosotros!

¡Ténganme! Es que... ¡Ténganme!

Yo lo mato..."

[En efecto, la tuvieron]

In [ ]:
# Dibujos de resumen
ax1 = WA_Person.plot(x="LastSeen", y="Hora", style='.', color='b', legend=False)
WA_Corp.plot(x="LastSeen", y="Hora", style='.', color='r', ax=ax1, legend=False)
plt.title("Hora de Desconexión VS. Fecha (azul=personal; rojo=corporativo)")
plt.show()

TCred["Valor"].plot()
plt.title("Gastos Tarjeta Credito Junio a Agosto")
plt.show()

Cel["DeptoMuni"].replace(to_replace={'23672':1, '11001':0}, inplace=True)
Cel.set_index("Hora", inplace=True)
Cel["DeptoMuni"].plot()
plt.title("Llamadas con origen en Bogota (0) y con origen en San Antero (1)")
plt.show()

La empezó a enredar arrancando julio...

A fin de mes le compró paseo...

Se la llevó empezando agosto... 3 millones por un fin de semana que ni puente era.

¡Y ES TAN CONCHUDO QUE NI LE VOLVIÓ A COQUETEAR!